# 3D UNet


In [ ]:
# hide
import sys
sys.path.append("..")
from nbdev.showdoc import *

In [ ]:
# default_exp models.unet
# export 
from fastai.basics import *
from fastai.layers import *
import torchvision, torch
from warnings import warn
from faimed3d.models.resnet import *
from faimed3d.models.deeplabv3 import *

## Building Blocks


### Convolutional BlocKs
#### DoubleConv
Standard double Convulutional-Block for UNet adapted for 3D.


In [ ]:
# export
class DoubleConv(nn.Module):
    """(convolution => [BN] => ReLU) * 2"""

    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        if not mid_channels:
            mid_channels = out_channels
        self.double_conv = nn.Sequential(
            nn.Conv3d(in_channels, mid_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(mid_channels),
            nn.ReLU(inplace=True),
            nn.Conv3d(mid_channels, out_channels, kernel_size=3, padding=1),
            nn.BatchNorm3d(out_channels),
            nn.ReLU(inplace=True)
        )

    def forward(self, x):
        return self.double_conv(x)


In [ ]:
DoubleConv(32, 64)(torch.randn(2, 32, 10, 50, 50)).size()

torch.Size([2, 64, 10, 50, 50])

In [ ]:
# export 
class DoubleResBlock(nn.Module):
    
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super().__init__()
        
        if not mid_channels: 
            mid_channels = in_channels//2
        
        self.res_block = nn.Sequential(
            # 1st Block
            nn.Conv3d(in_channels, mid_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm3d(mid_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),

            nn.Conv3d(mid_channels, mid_channels, kernel_size = 3, stride = 1, padding = 1, bias = False, groups = 4),
            nn.BatchNorm3d(mid_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),

            nn.Conv3d(mid_channels, mid_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm3d(mid_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),      
            nn.ReLU(inplace=True),            
            
            
            # 2nd Block
            nn.Conv3d(mid_channels, out_channels, kernel_size = 1, stride = 1, bias = False, groups = 16),
            nn.BatchNorm3d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),

            nn.Conv3d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias = False, groups = 16),
            nn.BatchNorm3d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),

            nn.Conv3d(out_channels, out_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm3d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)      
        )
    
    def forward(self, x):
        return self.res_block(x)

In [ ]:
DoubleResBlock(32, 64)(torch.randn(2, 32, 10, 50, 50)).size()

torch.Size([2, 64, 10, 50, 50])

## Down Block

In [ ]:
# export
class Down(nn.Module):
    """Downscaling with maxpool then resblock"""

    def __init__(self, in_channels, out_channels, block=DoubleConv):
        super().__init__()
        self.maxpool_conv = nn.Sequential(
            nn.MaxPool3d(kernel_size = (1, 2, 2)),
            block(in_channels, out_channels)
        )

    def forward(self, x):
        return self.maxpool_conv(x)

In [ ]:
Down(32,64,DoubleResBlock)(torch.randn(2, 32, 10, 50, 50)).size()

torch.Size([2, 64, 10, 25, 25])

### Upscaling Block

In [ ]:
# export
class Up(nn.Module):
    """Upscaling then double conv"""

    def __init__(self, in_channels_1, in_channels_2, out_channels = None, trilinear=True, block=DoubleConv):
        super().__init__()    
        
        if out_channels is None: out_channels = in_channels_1 // 2
        if trilinear:
            self.up = nn.Upsample(scale_factor=2, mode='trilinear', align_corners=True)
            self.conv = DoubleConv(in_channels_1+in_channels_2, out_channels)
        else:
            self.up = nn.ConvTranspose3d(in_channels_1 , in_channels_1 // 2, kernel_size=2, stride=2)
            self.conv = DoubleConv(in_channels_2+in_channels_1//2, out_channels)

    def forward(self, x1, x2):
        
        x1 = self.up(x1)
        # input is CHW
        diffZ = x2.size()[2] - x1.size()[2]
        diffY = x2.size()[3] - x1.size()[3]
        diffX = x2.size()[4] - x1.size()[4]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2, 
                        diffZ // 2, diffZ - diffZ // 2])

        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)

In [ ]:
Up(256, 128, 128, trilinear = True, block=DoubleResBlock)(torch.randn(10, 256, 1, 5, 5),torch.randn(10, 128, 2, 10, 10)).shape

torch.Size([10, 128, 2, 10, 10])

In [ ]:
Up(128, 256, 128, trilinear = True, block=DoubleResBlock)(torch.randn(10, 128, 1, 5, 5),torch.randn(10, 256, 2, 10, 10)).shape

torch.Size([10, 128, 2, 10, 10])

In [ ]:
Up(256, 128, 128, trilinear = False, block=DoubleResBlock)(torch.randn(10, 256, 1, 5, 5),torch.randn(10, 128, 2, 10, 10)).shape

torch.Size([10, 128, 2, 10, 10])

In [ ]:
Up(128, 256, 128, trilinear = False, block=DoubleResBlock)(torch.randn(10, 128, 1, 5, 5),torch.randn(10, 256, 2, 10, 10)).shape

torch.Size([10, 128, 2, 10, 10])

### Out Layers

In [ ]:
# export
class OutConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(OutConv, self).__init__()
        self.conv = nn.Conv3d(in_channels, out_channels, kernel_size=1)
    #    self.act = nn.Softmax(dim=1)    

    def forward(self, x):
        return self.conv(x)
    
class OutDuoubleRes(nn.Module):
    def __init__(self, in_channels, out_channels, mid_channels=None):
        super(OutDuoubleRes, self).__init__()
        if mid_channels is None: mid_channels = in_channels // 4
        self.conv =  nn.Sequential(
            # 1st Block
            nn.Conv3d(in_channels, mid_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm3d(mid_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),

            nn.Conv3d(mid_channels, mid_channels, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.BatchNorm3d(mid_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),

            nn.Conv3d(mid_channels, mid_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm3d(mid_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),      
            nn.ReLU(inplace=True),            
            
            
            # 2nd Block
            nn.Conv3d(mid_channels, out_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm3d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),

            nn.Conv3d(out_channels, out_channels, kernel_size = 3, stride = 1, padding = 1, bias = False),
            nn.BatchNorm3d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
            nn.ReLU(inplace=True),

            nn.Conv3d(out_channels, out_channels, kernel_size = 1, stride = 1, bias = False),
            nn.BatchNorm3d(out_channels, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)      
        )
   #     self.act = nn.Softmax(dim=1)    
            
    def forward(self, x):
        return self.conv(x)


## Models

### Base Classes

UNet architecture as initially described by [Ronneberger et al](https://arxiv.org/abs/1505.04597).

In [ ]:
# export
class AbstractUNet3D(nn.Module):
    "Abstract base class of UNet, should be subclassed by all UNets"
    def __init__(self):
        super(AbstractUNet3D, self).__init__()
    
    def forward(self, input):
        x1, x2, x3, x4, x5 = self.encoder(input)
        x = self.up1(x5, x4)
        x = self.up2(x, x3)
        x = self.up3(x, x2)
        x = self.up4(x, x1)
        logits = self.outc(x)
        return logits

### 3D UResNet

3D version of the original UNet with ResNet3D Encoder

In [ ]:
# export
class UResNet3D(AbstractUNet3D):
    def __init__(self, backbone, n_channels, n_classes, trilinear=False):
        super(UResNet3D, self).__init__()
        self.encoder=build_backbone(backbone, n_channels=n_channels, output_stride=16, BatchNorm=nn.BatchNorm3d)
        
        ## BasicBlock leads to smaller channelsize, how to correct this wisely??
        in_channels = 512 if isinstance(self.encoder.layer1[0], BasicBlock3d) else 2048
        
        self.up1 = Up(in_channels, in_channels // 2, trilinear=trilinear, block=DoubleResBlock)
        in_channels = in_channels // 2
        self.up2 = Up(in_channels, in_channels // 2, trilinear=trilinear, block=DoubleResBlock)
        in_channels = in_channels // 2
        self.up3 =Up(in_channels, in_channels // 2, trilinear=trilinear, block=DoubleResBlock)
        in_channels = in_channels // 2
        self.up4 = Up(in_channels, in_channels // 2, trilinear=trilinear, block=DoubleResBlock)
        self.outc = OutDuoubleRes(in_channels // 2, n_classes)
        

class UResNet3D_2(AbstractUNet3D):
    def __init__(self, n_channels, n_classes, trilinear=True, block=DoubleResBlock):
        super(UResNet3D_2, self).__init__()

        self.inc = DoubleConv(n_channels, 64)
        self.down1 = Down(64, 128, block=block)
        self.down2 = Down(128, 256, block=block)
        self.down3 = Down(256, 512, block=block)
        self.down4 = Down(512, 1024, block=block)
        self.up1 = Up(1024, 512, trilinear=trilinear, block=block)
        self.up2 = Up(512, 256, trilinear=trilinear, block=block)
        self.up3 = Up(256, 128, trilinear=trilinear, block=block)
        self.up4 = Up(128, 64, trilinear=trilinear, block=block)
        self.outc = OutDuoubleRes(64, n_classes)
        
    def encoder(self, x): 
        x1 = self.inc(x)
        x2 = self.down1(x1)
        x3 = self.down2(x2)
        x4 = self.down3(x3)
        x5 = self.down4(x4)
        return x1, x2, x3, x4, x5

In [ ]:
UResNet3D_2(n_channels=5, n_classes=2, trilinear=True)(torch.randn(5, 5, 5, 50, 50)).size()

torch.Size([5, 2, 5, 50, 50])

In [ ]:
UResNet3D_2(n_channels=5, n_classes=2, trilinear=True)

UResNet3D_2(
  (inc): DoubleConv(
    (double_conv): Sequential(
      (0): Conv3d(5, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
      (4): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): ReLU(inplace=True)
    )
  )
  (down1): Down(
    (maxpool_conv): Sequential(
      (0): MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2), padding=0, dilation=1, ceil_mode=False)
      (1): DoubleResBlock(
        (res_block): Sequential(
          (0): Conv3d(64, 32, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (1): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU(inplace=True)
          (3): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1),

### Double UNet 
as describet by [Debesh et al.](https://arxiv.org/pdf/2006.04868.pdf), adapted for 3D.   
The original architecture as published: 

![Double UNet Architecture](https://raw.githubusercontent.com/DebeshJha/2020-CBMS-DoubleU-Net/master/img/DoubleU-Net.png)

In [ ]:
# export
class DoubleUNet3D(nn.Module):
    def __init__(self, backbone, n_channels, n_classes, trilinear=False):
        store_attr()
        super(DoubleUNet3D, self).__init__()
        self.encoder1=build_backbone(backbone, n_channels=self.n_channels, output_stride=16, BatchNorm=nn.BatchNorm3d)
        self.encoder2=build_backbone(backbone, n_channels=self.n_channels, output_stride=16, BatchNorm=nn.BatchNorm3d)
        
        ## BasicBlock leads to smaller channelsize, how to correct this wisely??
        in_channels = 512 if isinstance(self.encoder1.layer1[0], BasicBlock3d) else 2048
        
        self.aspp1 = ASPP(in_channels, 8, nn.BatchNorm3d)
        self.aspp2 = ASPP(in_channels, 8, nn.BatchNorm3d)
        
        self.up1a = Up(256, in_channels // 2, trilinear=self.trilinear)
        self.up1b = Up(256, in_channels, trilinear=self.trilinear)
        
        in_channels = in_channels // 2
        self.up2a = Up(128, in_channels // 2, trilinear=self.trilinear)
        self.up2b = Up(128, in_channels, trilinear=self.trilinear)
        
        in_channels = in_channels // 2
        self.up3a = Up(64, in_channels // 2, trilinear=self.trilinear)
        self.up3b = Up(64, in_channels, trilinear=self.trilinear)
        
        in_channels = in_channels // 2
        self.up4a = Up(32, in_channels // 2, trilinear=self.trilinear)
        self.up4b = Up(32, in_channels, trilinear=self.trilinear)
        
        self.outc1 = OutConv(16, self.n_classes)
        self.outc2 = OutConv(16, self.n_classes)
        self.outc3 = OutConv(self.n_classes * 2, self.n_classes)
        
    def forward(self, input):
        x1a, x2a, x3a, x4a, x5a = self.encoder1(input)
        x = self.aspp1(x5a)
        x = self.up1a(x, x4a)
        x = self.up2a(x, x3a)
        x = self.up3a(x, x2a)
        x = self.up4a(x, x1a)
        out1 = self.outc1(x)

        input = torch.stack((torch.mean(out1, 1), )*self.n_channels, 1) * input
        x1b, x2b, x3b, x4b, x5b = self.encoder2(input)
        x = self.aspp1(x5b)
        x = self.up1b(x, torch.cat((x4a,x4b),1))
        x = self.up2b(x, torch.cat((x3a,x3b),1))
        x = self.up3b(x, torch.cat((x2a,x2b),1))
        x = self.up4b(x, torch.cat((x1a,x1b),1))
        out2 = self.outc2(x)
        
        return self.outc3(torch.cat((out1, out2), 1))

In [ ]:
DoubleUNet3D(resnet18_3d, n_channels=1, n_classes=2, trilinear=False)(torch.randn(5, 1, 3, 30, 30)).size()

torch.Size([5, 2, 3, 30, 30])

In [ ]:
DoubleUNet3D(resnet18_3d, n_channels=1, n_classes=2, trilinear=True)(torch.randn(5, 1, 5, 30, 30)).size()

torch.Size([5, 2, 5, 30, 30])

In [ ]:
DoubleUNet3D(resnet50_3d, n_channels=10, n_classes=2, trilinear=False)(torch.randn(5, 10, 3, 30, 30)).size()

torch.Size([5, 2, 3, 30, 30])

In [ ]:
DoubleUNet3D(resnet50_3d, n_channels=1, n_classes=2, trilinear=True)(torch.randn(2, 1, 3, 30, 30)).size()

torch.Size([2, 2, 3, 30, 30])

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 01_basics.ipynb.
Converted 02_transforms.ipynb.
Converted 03_datablock.ipynb.
Converted 04_datasets.ipynb.
Converted 05a_models.modules.ipynb.
Converted 05b_models.alexnet.ipynb.
Converted 05b_models.deeplabv3.ipynb.
Converted 05b_models.resnet.ipynb.
Converted 05c_models.siamese.ipynb.
Converted 05c_models.unet.ipynb.
Converted 05d_models.losses.ipynb.
Converted 06_callback.ipynb.
Converted 99_tools.ipynb.
Converted index.ipynb.
